In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from warnings import filterwarnings
from sklearn.tree import DecisionTreeRegressor, export_text, plot_tree
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [20]:
# read data
data = pd.read_csv("insurance.csv")
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [21]:
# change "sex children smoker region" to category

cols = ["sex", "children", "smoker", "region"]
data[cols] = data[cols].astype("category")

In [22]:
q1 = np.quantile(data["bmi"], 0.25)
q3 = np.quantile(data["bmi"], 0.75)
iqr = q3 - q1

upper = q3 + 1.5 * iqr
upper_outliers = data[data["bmi"] > upper]["bmi"].values
data["bmi"] = data["bmi"].replace(upper_outliers, upper)

In [23]:
# split data to X and y
X = data.iloc[:, :-1]
y = data.iloc[:, -1:]

In [24]:
# split data to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, shuffle=True)


# split train to train and val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, shuffle=True)

**Normaliztaion**

In [25]:
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

# apply min max scaler to numeric columns
numeric_cols = ["age", "bmi"]

########## X scaler ##########
# fit transform on X train
X_scaler.fit(X_train[numeric_cols])
X_train[numeric_cols] = X_scaler.transform(X_train[numeric_cols])

# transform only on test and validate
X_test[numeric_cols] = X_scaler.transform(X_test[numeric_cols])
X_val[numeric_cols] = X_scaler.transform(X_val[numeric_cols])

########## y scaler ##########
# fit transform on y train
y_scaler.fit(y_train)
y_train = y_scaler.transform(y_train)

# transform only on test and validate
y_test = y_scaler.transform(y_test)
y_val= y_scaler.transform(y_val)

**Label** **Encoding**

In [26]:
# Label encoding
le = preprocessing.LabelEncoder()

col_to_labelEncoding = "sex smoker region".split(" ")
for column in col_to_labelEncoding:
  # fit and transform on train
  le.fit(X_train[column])
  X_train[column] =  le.transform(X_train[column])
  # transform
  X_test[column] =  le.transform(X_test[column])
  X_val[column] =  le.transform(X_val[column])

In [27]:
X_train

,age,sex,bmi,children,smoker,region
237,0.282609,1,0.715927,2,0,2
613,0.347826,0,0.097032,3,0,0
877,0.326087,1,0.557932,5,0,2
226,0.217391,1,0.705394,0,0,2
482,0.000000,0,0.491222,0,0,2
...,...,...,...,...,...,...
1166,0.847826,1,0.779125,0,0,2
86,0.847826,0,0.485158,0,1,1
138,0.782609,0,0.508778,3,0,2
558,0.369565,0,0.579157,3,1,1


**Model**

In [30]:
model = DecisionTreeRegressor(random_state=0)
model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [32]:
pred_y_train = model.predict(X_train)
mean_absolute_error(y_train, pred_y_train)

0.00018778182222550254

In [34]:
pred_y_val = model.predict(X_val)
mean_absolute_error(y_val, pred_y_val)

0.06806296128764487